<a href="https://colab.research.google.com/github/vamshikrishna1370/Image_processing_kriti_2017/blob/master/supresV_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import glob
import subprocess
import os
from PIL import Image
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback
import wandb
from wandb.keras import WandbCallback

run = wandb.init(project='superres',name="InitnFinal5X5_adam_mse")#,run_id='hello')
# wandb.create_run(project='superres', username='vkreddy', run_id='Conv_size_InF_5')
# api = wandb.Api()
# run = api.create_run("vkreddy/superres/conv_sizeInF_5")
config = run.config


val_dir = 'data/test'
train_dir = 'data/train'

# automatically get the data if it doesn't exist
if not os.path.exists("data"):
    print("Downloading flower dataset...")
    subprocess.check_output(
        "mkdir data && curl https://storage.googleapis.com/wandb/flower-enhance.tar.gz | tar xzf - -C data", shell=True)





In [0]:
# !pip install wandb


In [0]:
def image_generator(batch_size, img_dir):
    """A generator that returns small images and large images.  DO NOT ALTER the validation set"""
    input_filenames = glob.glob(img_dir + "/*-in.jpg")
    counter = 0
    random.shuffle(input_filenames)
    while True:
        small_images = np.zeros(
            (batch_size, config.input_width, config.input_height, 3))
        large_images = np.zeros(
            (batch_size, config.output_width, config.output_height, 3))
        if counter+batch_size >= len(input_filenames):
            counter = 0
        for i in range(batch_size):
            img = input_filenames[counter + i]
            small_images[i] = np.array(Image.open(img)) / 255.0
            large_images[i] = np.array(
                Image.open(img.replace("-in.jpg", "-out.jpg"))) / 255.0
        yield (small_images, large_images)
        counter += batch_size


def perceptual_distance(y_true, y_pred):
    """Calculate perceptual distance, DO NOT ALTER"""
    y_true *= 255
    y_pred *= 255
    rmean = (y_true[:, :, :, 0] + y_pred[:, :, :, 0]) / 2
    r = y_true[:, :, :, 0] - y_pred[:, :, :, 0]
    g = y_true[:, :, :, 1] - y_pred[:, :, :, 1]
    b = y_true[:, :, :, 2] - y_pred[:, :, :, 2]

    return K.mean(K.sqrt((((512+rmean)*r*r)/256) + 4*g*g + (((767-rmean)*b*b)/256)))


class ImageLogger(Callback):
    def on_epoch_end(self, epoch, logs):
        preds = self.model.predict(in_sample_images)
        in_resized = []
        for arr in in_sample_images:
            # Simple upsampling
            in_resized.append(arr.repeat(8, axis=0).repeat(8, axis=1))
        wandb.log({
            "examples": [wandb.Image(np.concatenate([in_resized[i] * 255, o * 255, out_sample_images[i] * 255], axis=1)) for i, o in enumerate(preds)]
        }, commit=False)




In [0]:
config.num_epochs = 50
config.batch_size = 64
config.input_height = 32
config.input_width = 32
config.output_height = 256
config.output_width = 256

config.steps_per_epoch = len(
    glob.glob(train_dir + "/*-in.jpg")) // config.batch_size
config.val_steps_per_epoch = len(
    glob.glob(val_dir + "/*-in.jpg")) // config.batch_size


val_generator = image_generator(config.batch_size, val_dir)
in_sample_images, out_sample_images = next(val_generator)



In [5]:
model = Sequential()
model.add(layers.Conv2D(3, (5, 5), activation='relu', padding='same',
                        input_shape=(config.input_width, config.input_height, 3)))
model.add(layers.UpSampling2D())
model.add(layers.Conv2D(3, (3, 3), activation='relu', padding='same'))
model.add(layers.UpSampling2D())
model.add(layers.Conv2D(3, (3, 3), activation='relu', padding='same'))
model.add(layers.UpSampling2D())
model.add(layers.Conv2D(3, (5, 5), activation='relu', padding='same'))

# DONT ALTER metrics=[perceptual_distance]


W0812 17:42:56.272908 139682311153536 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
model.compile(optimizer='adam', loss='mse',
              metrics=[perceptual_distance])

model.fit_generator(image_generator(config.batch_size, train_dir),
                    steps_per_epoch=config.steps_per_epoch,
                    epochs=config.num_epochs, callbacks=[
                        ImageLogger(), WandbCallback()],
                    validation_steps=config.val_steps_per_epoch,
                    validation_data=val_generator)

Epoch 1/50
156/156 [==============================] - 21s 135ms/step - loss: 0.0887 - perceptual_distance: 183.9177 - val_loss: 0.0393 - val_perceptual_distance: 126.7149
Epoch 2/50
156/156 [==============================] - 18s 118ms/step - loss: 0.0342 - perceptual_distance: 116.3130 - val_loss: 0.0313 - val_perceptual_distance: 111.6615
Epoch 3/50
156/156 [==============================] - 19s 119ms/step - loss: 0.0285 - perceptual_distance: 106.0588 - val_loss: 0.0271 - val_perceptual_distance: 103.6651
Epoch 4/50
156/156 [==============================] - 19s 120ms/step - loss: 0.0255 - perceptual_distance: 99.9221 - val_loss: 0.0250 - val_perceptual_distance: 99.3064
Epoch 5/50
156/156 [==============================] - 19s 120ms/step - loss: 0.0239 - perceptual_distance: 96.3111 - val_loss: 0.0237 - val_perceptual_distance: 96.1363
Epoch 6/50
156/156 [==============================] - 19s 120ms/step - loss: 0.0228 - perceptual_distance: 93.7355 - val_loss: 0.0229 - val_perceptua